# Day 3-2: 분석가를 위한 표준 라이브러리 II - 정답

**학습 목표:**
- 파일/경로 제어: os, glob, shutil로 파일 자동 정리
- 데이터 검색: re (정규표현식) 모듈로 텍스트 패턴 추출
- 날짜/시간: datetime 모듈 활용

**실무 활용:**
- 수백 개의 엑셀 파일명을 일괄 변경
- 비정형 텍스트에서 이메일/전화번호 추출
- 날짜 기반 파일 정리 및 필터링

## 1. 파일/경로 제어 기초

### 1.1 os 모듈 기본

파일 시스템 작업의 핵심 모듈입니다.

In [ ]:
import os

# 현재 작업 디렉토리 확인
print("현재 디렉토리:", os.getcwd())

# 경로 결합 (운영체제에 맞는 구분자 자동 사용)
data_path = os.path.join('data', 'reports', '2024.xlsx')
print("경로:", data_path)

# 경로 분리
directory, filename = os.path.split(data_path)
print(f"디렉토리: {directory}, 파일명: {filename}")

# 파일명과 확장자 분리
name, ext = os.path.splitext(filename)
print(f"이름: {name}, 확장자: {ext}")

### 1.2 pathlib 모듈 (최신 방식)

객체 지향적인 경로 처리 방법입니다.

In [ ]:
from pathlib import Path

# Path 객체 생성
path = Path('data') / 'reports' / '2024.xlsx'
print("경로:", path)

# 경로 정보 추출
print("부모 디렉토리:", path.parent)
print("파일명:", path.name)
print("확장자:", path.suffix)
print("확장자 제외 이름:", path.stem)

# 경로 존재 확인
print("존재 여부:", path.exists())
print("파일 여부:", path.is_file())
print("디렉토리 여부:", path.is_dir())

### 1.3 glob 모듈 - 파일 패턴 검색

와일드카드를 사용하여 파일을 찾습니다.

In [ ]:
import glob

# 예제 파일 생성 (실습용)
example_dir = Path('temp_files')
example_dir.mkdir(exist_ok=True)

# 샘플 파일 생성
for i in range(3):
    (example_dir / f'report_2024_{i+1:02d}.xlsx').touch()
    (example_dir / f'data_2024_{i+1:02d}.csv').touch()

# glob 패턴 사용
print("모든 엑셀 파일:")
for file in glob.glob('temp_files/*.xlsx'):
    print(f"  {file}")

print("\n2024로 시작하는 모든 파일:")
for file in glob.glob('temp_files/*2024*'):
    print(f"  {file}")

# pathlib의 glob (더 편리함)
print("\nPathlib glob - CSV 파일:")
for file in example_dir.glob('*.csv'):
    print(f"  {file.name}")

### 1.4 shutil 모듈 - 파일 작업

파일 복사, 이동, 삭제 등의 고수준 작업입니다.

In [ ]:
import shutil

# 디렉토리 생성
backup_dir = Path('temp_backup')
backup_dir.mkdir(exist_ok=True)

# 파일 복사
source = example_dir / 'report_2024_01.xlsx'
dest = backup_dir / 'report_2024_01_backup.xlsx'
shutil.copy(source, dest)
print(f"복사 완료: {dest}")

# 파일 이동 (이름 변경에도 사용)
old_name = example_dir / 'data_2024_01.csv'
new_name = example_dir / 'dataset_2024_01.csv'
shutil.move(old_name, new_name)
print(f"이동 완료: {new_name}")

# 디렉토리 전체 복사
# shutil.copytree(example_dir, 'temp_files_backup')

# 정리
shutil.rmtree(backup_dir)
print("백업 디렉토리 삭제 완료")

## 2. 정규표현식 (Regular Expression)

### 2.1 정규표현식 기초

텍스트 패턴을 찾고 처리하는 강력한 도구입니다.

In [ ]:
import re

# 기본 패턴 매칭
text = "연락처: 010-1234-5678, 이메일: user@example.com"

# 전화번호 찾기
phone_pattern = r'\d{3}-\d{4}-\d{4}'
phone = re.search(phone_pattern, text)
if phone:
    print("전화번호:", phone.group())

# 이메일 찾기
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
email = re.search(email_pattern, text)
if email:
    print("이메일:", email.group())

### 2.2 정규표현식 패턴 요소

| 패턴 | 설명 | 예시 |
|------|------|------|
| `\d` | 숫자 (0-9) | `\d{3}` → 123 |
| `\w` | 단어 문자 (a-z, A-Z, 0-9, _) | `\w+` → hello |
| `\s` | 공백 문자 | `\s+` → 공백들 |
| `.` | 임의의 문자 | `a.c` → abc, a1c |
| `+` | 1회 이상 반복 | `\d+` → 123 |
| `*` | 0회 이상 반복 | `\d*` → "", 123 |
| `?` | 0회 또는 1회 | `\d?` → "", 1 |
| `{n}` | 정확히 n회 | `\d{3}` → 123 |
| `{n,m}` | n~m회 | `\d{2,4}` → 12, 123, 1234 |
| `[]` | 문자 집합 | `[abc]` → a, b, c |
| `^` | 문자열 시작 | `^Hello` |
| `$` | 문자열 끝 | `world$` |

In [ ]:
# 다양한 패턴 예제
sample_text = """
날짜: 2024-01-15, 2024-02-20
전화: 010-1234-5678, 02-9876-5432
이메일: admin@company.co.kr, user123@gmail.com
금액: 1,234,567원, ₩2,000,000
"""

# 날짜 찾기 (YYYY-MM-DD)
dates = re.findall(r'\d{4}-\d{2}-\d{2}', sample_text)
print("날짜:", dates)

# 모든 전화번호 찾기
phones = re.findall(r'\d{2,3}-\d{4}-\d{4}', sample_text)
print("전화번호:", phones)

# 모든 이메일 찾기
emails = re.findall(r'[\w.+-]+@[\w.-]+\.[\w]+', sample_text)
print("이메일:", emails)

# 숫자 (콤마 포함) 찾기
numbers = re.findall(r'[\d,]+', sample_text)
print("숫자:", numbers)

### 2.3 정규표현식 그룹 활용

괄호를 사용하여 패턴의 일부를 추출할 수 있습니다.

In [ ]:
# 이메일 분리 (사용자명, 도메인)
email_text = "Contact: john.doe@company.com, jane@example.co.kr"

# 그룹 사용
pattern = r'([\w.+-]+)@([\w.-]+\.[\w]+)'
matches = re.findall(pattern, email_text)

print("이메일 분석:")
for username, domain in matches:
    print(f"  사용자: {username}, 도메인: {domain}")

# 날짜 분리 (년, 월, 일)
date_text = "보고서 날짜: 2024-03-15, 2024-12-25"
date_pattern = r'(\d{4})-(\d{2})-(\d{2})'
date_matches = re.findall(date_pattern, date_text)

print("\n날짜 분석:")
for year, month, day in date_matches:
    print(f"  {year}년 {month}월 {day}일")

### 2.4 정규표현식 치환

패턴을 찾아 다른 텍스트로 바꿉니다.

In [ ]:
# 전화번호 형식 변경
text = "연락처: 010-1234-5678, 02-987-6543"

# 하이픈 제거
cleaned = re.sub(r'(\d{2,3})-(\d{3,4})-(\d{4})', r'\1\2\3', text)
print("하이픈 제거:", cleaned)

# 괄호 형식으로 변경
formatted = re.sub(r'(\d{2,3})-(\d{3,4})-(\d{4})', r'(\1) \2-\3', text)
print("괄호 형식:", formatted)

# 민감 정보 마스킹
masked = re.sub(r'(\d{3})-\d{4}-(\d{4})', r'\1-****-\2', text)
print("마스킹:", masked)

## 3. 날짜/시간 처리

### 3.1 datetime 기본

In [ ]:
from datetime import datetime, date, time, timedelta

# 현재 날짜와 시간
now = datetime.now()
print("현재:", now)
print("날짜만:", now.date())
print("시간만:", now.time())

# 특정 날짜 생성
specific_date = datetime(2024, 3, 15, 14, 30, 0)
print("\n특정 날짜:", specific_date)

# 날짜 요소 추출
print(f"년: {now.year}")
print(f"월: {now.month}")
print(f"일: {now.day}")
print(f"시: {now.hour}")
print(f"분: {now.minute}")
print(f"요일: {now.weekday()}")  # 0=월요일, 6=일요일

### 3.2 날짜 포맷팅과 파싱

In [ ]:
# 날짜를 문자열로 변환 (strftime)
now = datetime.now()

print("다양한 포맷:")
print(now.strftime("%Y-%m-%d"))           # 2024-03-15
print(now.strftime("%Y년 %m월 %d일"))      # 2024년 03월 15일
print(now.strftime("%Y-%m-%d %H:%M:%S")) # 2024-03-15 14:30:45
print(now.strftime("%Y%m%d"))            # 20240315

# 문자열을 날짜로 변환 (strptime)
date_str = "2024-03-15"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
print("\n파싱된 날짜:", date_obj)

# 다양한 형식 파싱
formats = [
    ("2024/03/15", "%Y/%m/%d"),
    ("15-03-2024", "%d-%m-%Y"),
    ("2024년 3월 15일", "%Y년 %m월 %d일"),
]

print("\n다양한 형식 파싱:")
for date_str, format_str in formats:
    parsed = datetime.strptime(date_str, format_str)
    print(f"{date_str} → {parsed}")

### 3.3 날짜 연산

**주요 포맷 코드:**
- `%Y`: 4자리 연도 (2024)
- `%m`: 2자리 월 (01-12)
- `%d`: 2자리 일 (01-31)
- `%H`: 24시간 형식 시간 (00-23)
- `%M`: 분 (00-59)
- `%S`: 초 (00-59)
- `%A`: 요일 전체 (Monday)
- `%B`: 월 이름 전체 (January)

In [ ]:
from datetime import timedelta

today = datetime.now()

# 날짜 더하기/빼기
tomorrow = today + timedelta(days=1)
yesterday = today - timedelta(days=1)
next_week = today + timedelta(weeks=1)
next_month = today + timedelta(days=30)

print("오늘:", today.strftime("%Y-%m-%d"))
print("내일:", tomorrow.strftime("%Y-%m-%d"))
print("어제:", yesterday.strftime("%Y-%m-%d"))
print("1주일 후:", next_week.strftime("%Y-%m-%d"))
print("30일 후:", next_month.strftime("%Y-%m-%d"))

# 날짜 차이 계산
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
diff = end_date - start_date

print(f"\n날짜 차이: {diff.days}일")
print(f"시간 차이: {diff.total_seconds() / 3600:.0f}시간")

### 3.4 날짜 범위 생성

In [ ]:
# 날짜 범위 생성 (pandas 없이)
start = datetime(2024, 1, 1)
end = datetime(2024, 1, 10)

print("날짜 범위:")
current = start
while current <= end:
    print(current.strftime("%Y-%m-%d %A"))
    current += timedelta(days=1)

# 월말 날짜 구하기
from calendar import monthrange

year, month = 2024, 2
last_day = monthrange(year, month)[1]
month_end = datetime(year, month, last_day)
print(f"\n2024년 2월 마지막 날: {month_end.strftime('%Y-%m-%d')}")

## 연습문제 정답

### 문제 1: 경로 정보 추출 ⭐

In [ ]:
from pathlib import Path

file_path = "data/reports/sales/2024/Q1_sales_report_final.xlsx"

# 1. Path 객체로 변환
path = Path(file_path)
print("1. Path 객체:", path)

# 2. 파일명 (확장자 포함)
filename = path.name
print("2. 파일명 (확장자 포함):", filename)

# 3. 파일명 (확장자 제외)
stem = path.stem
print("3. 파일명 (확장자 제외):", stem)

# 4. 확장자
suffix = path.suffix
print("4. 확장자:", suffix)

# 5. 상위 디렉토리 경로
parent = path.parent
print("5. 상위 디렉토리:", parent)

### 문제 2: 파일 검색 패턴 ⭐⭐

In [ ]:
from pathlib import Path

# 테스트 파일 생성
test_dir = Path('test_search')
test_dir.mkdir(exist_ok=True)

files = [
    'report_2024_01.xlsx',
    'report_2024_02.xlsx',
    'report_2023_12.xlsx',
    'data_2024_01.csv',
    'summary_2024.txt',
    'backup_2024_01.xlsx',
]

for f in files:
    (test_dir / f).touch()

# 1. 모든 .xlsx 파일
print("1. 모든 .xlsx 파일:")
for f in sorted(test_dir.glob('*.xlsx')):
    print(f"   {f.name}")

# 2. 2024년 파일만
print("\n2. 2024년 파일:")
for f in sorted(test_dir.glob('*2024*')):
    print(f"   {f.name}")

# 3. report로 시작하는 2024년 xlsx 파일
print("\n3. report로 시작하는 2024년 xlsx 파일:")
for f in sorted(test_dir.glob('report_2024*.xlsx')):
    print(f"   {f.name}")

# 정리
import shutil
shutil.rmtree(test_dir)

### 문제 3: 파일 일괄 이름 변경 ⭐⭐⭐

In [ ]:
from pathlib import Path
import re

# 테스트 파일 생성
rename_dir = Path('test_rename')
rename_dir.mkdir(exist_ok=True)

old_files = [
    'sales_20240115_report.xlsx',
    'data_20240220_final.csv',
    'summary_20231231.txt',
]

for f in old_files:
    (rename_dir / f).touch()

print("변경 전:")
for f in sorted(rename_dir.glob('*')):
    print(f"  {f.name}")

# 날짜 형식 변경 (YYYYMMDD → YYYY-MM-DD)
for file in rename_dir.glob('*'):
    # 파일명에서 YYYYMMDD 패턴 찾기
    new_name = re.sub(
        r'(\d{4})(\d{2})(\d{2})',  # YYYYMMDD 패턴
        r'\1-\2-\3',                # YYYY-MM-DD로 변경
        file.name
    )
    
    # 파일명이 변경되었으면 이름 변경
    if new_name != file.name:
        new_path = file.parent / new_name
        file.rename(new_path)
        print(f"\n변경: {file.name} → {new_name}")

print("\n변경 후:")
for f in sorted(rename_dir.glob('*')):
    print(f"  {f.name}")

# 정리
import shutil
shutil.rmtree(rename_dir)

### 문제 4: 텍스트에서 정보 추출 ⭐⭐⭐

In [ ]:
import re

sample_text = """
고객 정보 목록:

1. 김철수 (부장)
   - 연락처: 010-1234-5678
   - 이메일: kim.cs@company.co.kr
   - 부서: 영업팀

2. 이영희 (과장)
   - Tel: 010-9876-5432
   - Email: lee.yh@example.com
   - 부서: 마케팅팀

3. 박민수 (대리)
   - 전화: 02-1234-5678
   - 메일: park_ms@company.kr
   - 부서: 개발팀

담당자: admin@company.co.kr (02-9999-8888)
"""

# 1. 모든 전화번호 추출
phone_pattern = r'\d{2,3}-\d{3,4}-\d{4}'
phones = re.findall(phone_pattern, sample_text)
print("1. 전화번호:")
for phone in phones:
    print(f"   {phone}")

# 2. 모든 이메일 추출
email_pattern = r'[\w._%+-]+@[\w.-]+\.[\w]+'
emails = re.findall(email_pattern, sample_text)
print("\n2. 이메일:")
for email in emails:
    print(f"   {email}")

# 3. 딕셔너리 리스트로 정리
# 각 사람별로 정보를 매칭 (간단한 방법)
contacts = []
for i in range(min(len(phones), len(emails))):
    contacts.append({
        'phone': phones[i],
        'email': emails[i]
    })

print("\n3. 연락처 정보:")
for i, contact in enumerate(contacts, 1):
    print(f"   {i}. {contact}")

### 문제 5: 정규표현식 데이터 정제 ⭐⭐⭐⭐

In [ ]:
import re

phone_data = [
    "010-1234-5678",
    "01012345678",
    "010.1234.5678",
    "010 1234 5678",
    "(010) 1234-5678",
    "02-123-4567",
    "021234567",
    "02.123.4567",
]

def normalize_phone(phone):
    # 숫자만 추출
    digits = re.sub(r'\D', '', phone)
    
    # 길이에 따라 포맷팅
    if len(digits) == 11:  # 010-1234-5678
        return f"{digits[:3]}-{digits[3:7]}-{digits[7:]}"
    elif len(digits) == 10:  # 02-123-4567 또는 010-123-4567
        if digits.startswith('02'):
            return f"{digits[:2]}-{digits[2:5]}-{digits[5:]}"
        else:
            return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"
    elif len(digits) == 9:  # 02-123-4567
        return f"{digits[:2]}-{digits[2:5]}-{digits[5:]}"
    else:
        return phone  # 변경하지 않음

print("정규화된 전화번호:")
for phone in phone_data:
    normalized = normalize_phone(phone)
    print(f"{phone:20s} → {normalized}")

### 문제 6: 날짜 파싱과 포맷팅 ⭐⭐⭐⭐

In [ ]:
from datetime import datetime

date_strings = [
    "2024-01-15",
    "2024/01/15",
    "15-01-2024",
    "01/15/2024",
    "2024년 1월 15일",
    "20240115",
]

def parse_and_format(date_str):
    # 다양한 형식 시도
    formats = [
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%d-%m-%Y",
        "%m/%d/%Y",
        "%Y년 %m월 %d일",
        "%Y%m%d",
    ]
    
    for fmt in formats:
        try:
            dt = datetime.strptime(date_str, fmt)
            # 요일 이름 (한글)
            weekdays = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
            weekday = weekdays[dt.weekday()]
            return f"{dt.year}년 {dt.month:02d}월 {dt.day:02d}일 ({weekday})"
        except ValueError:
            continue
    
    return "파싱 실패"

print("날짜 변환 결과:")
for date_str in date_strings:
    result = parse_and_format(date_str)
    print(f"{date_str:20s} → {result}")

### 문제 7: 종합 - 로그 파일 분석 시스템 ⭐⭐⭐⭐⭐

In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
import re

# 테스트 로그 파일 생성
log_dir = Path('logs')
log_dir.mkdir(exist_ok=True)

# 샘플 로그 생성 (최근 7일)
for i in range(7):
    date = datetime.now() - timedelta(days=i)
    filename = f"system_{date.strftime('%Y%m%d')}.log"
    
    log_content = f"""
2024-01-{15+i:02d} 09:15:23 [INFO] User login: user{i}@example.com from 192.168.1.{100+i}
2024-01-{15+i:02d} 10:30:45 [ERROR] Failed login attempt: admin@test.com from 192.168.1.{200+i}
2024-01-{15+i:02d} 11:45:12 [INFO] File uploaded: report_{date.strftime('%Y%m%d')}.xlsx (2.5MB)
2024-01-{15+i:02d} 14:20:33 [WARNING] High memory usage: 85%
2024-01-{15+i:02d} 16:55:41 [INFO] User logout: user{i}@example.com
"""
    (log_dir / filename).write_text(log_content)

print("생성된 로그 파일:")
for f in sorted(log_dir.glob('*.log')):
    print(f"  {f.name}")

# 분석 시작
print("\n=" * 60)
print("로그 분석 결과")
print("=" * 60)

# 1. 최근 3일 로그 선택
today = datetime.now()
cutoff_date = today - timedelta(days=3)

recent_logs = []
for log_file in log_dir.glob('*.log'):
    # 파일명에서 날짜 추출 (system_YYYYMMDD.log)
    match = re.search(r'system_(\d{8})\.log', log_file.name)
    if match:
        file_date = datetime.strptime(match.group(1), '%Y%m%d')
        if file_date >= cutoff_date:
            recent_logs.append((file_date, log_file))

# 날짜순 정렬
recent_logs.sort(reverse=True)

# 2. 각 로그 파일 분석
for file_date, log_file in recent_logs:
    print(f"\n날짜: {file_date.strftime('%Y-%m-%d')}")
    
    # 로그 내용 읽기
    content = log_file.read_text()
    lines = content.strip().split('\n')
    
    # 총 로그 수
    total_logs = len([line for line in lines if line.strip()])
    print(f"- 총 로그: {total_logs}건")
    
    # 에러 개수
    errors = re.findall(r'\[ERROR\]', content)
    print(f"- 에러: {len(errors)}건")
    
    # 로그인 사용자
    login_pattern = r'User login: ([\w.@]+)'
    users = re.findall(login_pattern, content)
    if users:
        print(f"- 로그인 사용자: {', '.join(set(users))}")
    
    # 업로드 파일
    upload_pattern = r'File uploaded: ([\w_.-]+)'
    files = re.findall(upload_pattern, content)
    if files:
        print(f"- 업로드 파일: {', '.join(files)}")

print("\n" + "=" * 60)

# 정리
import shutil
shutil.rmtree(log_dir)
print("\n로그 파일 정리 완료")

## 실무 활용 예제

### 예제 1: 월별 보고서 파일 정리

In [ ]:
from pathlib import Path
import shutil
import re

# 파일을 월별 폴더로 정리하는 함수
def organize_reports_by_month(source_dir, target_dir):
    """
    파일명에서 날짜를 추출하여 YYYY-MM 폴더로 정리
    예: report_2024-01-15.xlsx → 2024-01/report_2024-01-15.xlsx
    """
    source = Path(source_dir)
    target = Path(target_dir)
    target.mkdir(exist_ok=True)
    
    # 날짜 패턴 (YYYY-MM-DD)
    date_pattern = r'(\d{4})-(\d{2})-(\d{2})'
    
    for file in source.glob('*'):
        if file.is_file():
            # 파일명에서 날짜 추출
            match = re.search(date_pattern, file.name)
            if match:
                year, month, day = match.groups()
                
                # 월별 폴더 생성
                month_dir = target / f"{year}-{month}"
                month_dir.mkdir(exist_ok=True)
                
                # 파일 이동
                dest = month_dir / file.name
                shutil.copy(file, dest)
                print(f"Moved: {file.name} → {month_dir.name}/")

# 예제 실행
# organize_reports_by_month('reports', 'organized_reports')

### 예제 2: 이메일 주소 검증 및 정제

In [ ]:
import re

def validate_and_clean_emails(text):
    """
    텍스트에서 유효한 이메일만 추출하고 정제
    """
    # 이메일 패턴
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    
    # 모든 이메일 찾기
    emails = re.findall(email_pattern, text)
    
    # 소문자로 변환 및 중복 제거
    cleaned_emails = sorted(set(email.lower() for email in emails))
    
    # 도메인별 분류
    by_domain = {}
    for email in cleaned_emails:
        domain = email.split('@')[1]
        if domain not in by_domain:
            by_domain[domain] = []
        by_domain[domain].append(email)
    
    return cleaned_emails, by_domain

# 테스트
sample = """
Contact: John@Example.COM, jane@test.com, admin@Example.com
Support: support@company.co.kr, Help@company.CO.KR
"""

emails, by_domain = validate_and_clean_emails(sample)
print("정제된 이메일:", emails)
print("\n도메인별:")
for domain, email_list in by_domain.items():
    print(f"  {domain}: {email_list}")

### 예제 3: 날짜 범위로 파일 필터링

In [ ]:
from pathlib import Path
from datetime import datetime, timedelta
import re

def filter_files_by_date_range(directory, start_date, end_date, pattern=r'(\d{4}-\d{2}-\d{2})'):
    """
    날짜 범위에 해당하는 파일만 필터링
    """
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    
    filtered_files = []
    
    for file in Path(directory).glob('*'):
        if file.is_file():
            # 파일명에서 날짜 추출
            match = re.search(pattern, file.name)
            if match:
                file_date = datetime.strptime(match.group(1), '%Y-%m-%d')
                
                # 날짜 범위 체크
                if start <= file_date <= end:
                    filtered_files.append(file)
    
    return sorted(filtered_files)

# 테스트
# files = filter_files_by_date_range('reports', '2024-01-01', '2024-01-31')
# for f in files:
#     print(f.name)

## 정리

### 핵심 요약

1. **파일/경로 제어**
   - `pathlib.Path`: 현대적인 경로 처리
   - `glob`: 패턴 기반 파일 검색
   - `shutil`: 파일 복사/이동/삭제

2. **정규표현식 (re)**
   - `re.search()`: 패턴 찾기
   - `re.findall()`: 모든 매치 추출
   - `re.sub()`: 패턴 치환
   - 그룹 `()`으로 부분 추출

3. **날짜/시간 (datetime)**
   - `strftime()`: 날짜 → 문자열
   - `strptime()`: 문자열 → 날짜
   - `timedelta`: 날짜 연산

### 실무 활용 시나리오

- ✅ 대량의 파일 일괄 처리
- ✅ 비정형 데이터에서 정보 추출
- ✅ 날짜 기반 데이터 필터링
- ✅ 로그 파일 분석
- ✅ 자동화된 파일 정리 시스템